In [1]:
# Collect the metadata into XML files

import requests

PAGE_SIZE = 100

def search(name, collection_url):
    root, scope = collection_url.split('/handle/')
    for i in range(12):
        params = {
            'query': 'dc.date.issued:[2020 TO 2022]',
            'scope': scope,
            'format': 'kkf',
            'rpp': PAGE_SIZE,
            'start': i * PAGE_SIZE,
            'sort_by': 3
        }
        url = root + '/open-search/'
        r = requests.get(url, params=params)
        print(root, scope, r.url)
        outfn = f'{name}-page{i+1:02d}.out'
        xmlfn = f'{name}-page{i+1:02d}.xml'
        with open(outfn, 'wb') as outf:
            for chunk in r.iter_content(chunk_size=1024):
                outf.write(chunk)
        ! xmllint --format {outfn} >{xmlfn}
        ! rm {outfn}


search('abo-theses', 'https://www.doria.fi/handle/10024/87831')

https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=0&sort_by=3
https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=100&sort_by=3
https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=200&sort_by=3
https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=300&sort_by=3
https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=400&sort_by=3
https://www.doria.fi 10024/87831 https://www.doria.fi/open-search/?query=dc.date.issued%3A%5B2020+TO+2022%5D&scope=10024%2F87831&format=kkf&rpp=100&start=500&sort

In [2]:
import glob
from lxml import etree
import skosmos_client
import functools

finto = skosmos_client.SkosmosClient()

@functools.lru_cache(maxsize=10000)
def lookup_yso(label, lang):
    if label is None:
        return None

    try:
        lookup_results = finto.lookup('yso', label, lang)
        return lookup_results[0]['uri']
    except ValueError:
        return None        

def infer_language(language, qualifier):
    if language in ('fi', 'sv', 'en'):
        return language
    if qualifier == 'yso/fin':
        return 'fi'
    if qualifier == 'yso/swe':
        return 'sv'
    return None  # any language



In [3]:
lookup_fixes = {
    'placeringar@sv': 'placeringar (ekonomi)@sv',
    'sijoitukset@fi': 'sijoitukset (talous)@fi',
    'peruskoulu@sv': 'peruskoulu@fi',
    'asenteet@sv': 'asenteet@fi',
    'värden@sv': 'värden (uppfattningar)@sv',
    'entrepenörskap@sv': 'entreprenörskap@sv',
    'arvot@fi': 'arvot (käsitykset)@fi',
    'dispersions@en': 'dispersions (mixtures)@en',
    'dispersioner@sv': 'dispersioner (blandningar)@sv',
    'dispersiot@fi': 'dispersiot (seokset)@fi',
    'opinioner@sv': 'åsikter@sv',
    'könsskilnader@sv': 'könsskillnader@sv',
    'fyysinen aktiivisuus@sv': 'fyysinen aktiivisuus@fi',
    'physical activiness@en': 'physical activeness@en',
    'motivation (mental object)@en': 'motivation (mental objects)@en',
    'slöjdpedagogik (vetenskap)@sv': 'slöjdpedagogik@sv',
    'positiivinen pegagogiikka@fi': 'positiivinen pedagogiikka@fi',
    'förbybara energikällor@sv': 'förnybara energikällor@sv',
    'startup-companies@en': 'startup companies@en',
    'international corporations@sv': 'international corporations@en',
    'Kristallnatt@sv': 'Kristallnatten@sv',
    'processtryrning@sv': 'processtyrning@sv',
    'utvrändhet@sv': 'utbrändhet@sv',
    'jämförände forskning@sv': 'jämförande forskning@sv',
    'opiskelijaelämää@fi': 'opiskelijaelämä@fi',
    'studentlive@sv': 'studentliv@sv',
    'socioekonominen asema@fi': 'sosioekonominen asema@fi',
    'religiositet@fi': 'religiositet@fi',
    'politik@fi': 'politiikka@fi',
    'rektor@sv': 'rektorer@sv',
    'talterapeuter@fi': 'talterapeuter@sv',
    'dokumenttifilmit@fi': 'dokumenttielokuvat@fi',
    'sleep dept@en': 'sleep debt@en',
    'nätverk@fi': 'nätverk@sv',
    'retorikka@fi': 'retoriikka@fi',
    'marinbiologi@fi': 'marinbiologi@sv',
    'ïnitiaatiot@fi': 'initiaatiot@fi',
    'biomass@en': 'biomass (ecology)@en',
    'biomassa@sv': 'biomass (ecology)@en',
    'biomassa@fi': 'biomass (ecology)@en',
    'funktioner (aktivitet)@sv': 'funktioner (aktiviteter)@sv',
    'verksamhet för bevarande av arbetsförmåga@sv': 'verksamhet för bevarande av arbetsförmågan@sv',
    'erilaiset oppijat@sv': 'erilaiset oppijat@fi',
    'vihreä liike@sv': 'vihreä liike@fi',
    'systematisk litteraturöversikt@sv': 'systematiska litteraturöversikter@sv',
    'polittiiset kriisit@fi': 'poliittiset kriisit@fi',
    'politiskt instabilitet@sv': 'politisk instabilitet@sv',
    'företag@en': 'företag@sv',
    'effektivitet@en': 'effektivitet@sv',
    'företagsköp@en': 'företagsköp@sv',
    'tillämpningsprogram@en': 'tillämpningsprogram@sv',
    'kundrelation@en': 'kundrelation@sv',
    'kundlojalitet@en': 'kundlojalitet@sv',
    'röntgenundersökningar@sv': 'röntgenundersökning@sv',
    'översköterska@sv': 'överskötare@sv',
    'viral proteines@en': 'viral proteins@en',
    'automation@fi': 'automation@sv',
    'arbetsförhållanden@sv': 'arbetsförhållanden (omständigheter)@sv',
    'etäytyö@fi': 'etätyö@fi',
    'samhälle@sv': 'samhällen@sv',
    'översättning (verksamhet)@fi': 'översättning (verksamhet)@sv',
    'datasäkerhet@fi': 'datasäkerhet@sv',
    'alkoholi (päihteet)@sv': 'alkoholi (päihteet)@fi',
    'alkoholinkäyttö@sv': 'alkoholinkäyttö@fi',
    'pandemita@fi': 'pandemiat@fi',
    'perheelämä@fi': 'perhe-elämä@fi',
    'veisto@fi': 'puunveisto@fi',
    'huonokuuluisuus@fi': 'huonokuuloisuus@fi',
    'maematiikka@fi': 'matematiikka@fi',
    'työn kuomittavuus@fi': 'työn kuormittavuus@fi',
    'kvalitattiivinen tuktimus@fi': 'kvalitatiivinen tutkimus@fi',
    'chefer@sv': 'förmän@sv',
    'arbetarsskydd@sv': 'arbetarskydd@sv',
    'skolo (läroanstalter)@sv': 'skolor (läroanstalter)@sv',
    'experiences@en': 'experiences (knowledge)@en',
    'Tredje riket@sv': 'Tredje riket (nazism)@sv',
    'kolmas valtakunta@fi': 'kolmas valtakunta (kansallissosialismi)@fi',
    'carbon footprint@sv': 'carbon footprint@en',
    'carbon dioxide@sv': 'carbon dioxide@en',
    'koldioxid@fi': 'koldioxid@sv',
    'moniaisuus@fi': 'moninaisuus@fi',
    'sportjournalist@sv': 'sportjournalister@sv',
    'kärnkraftverk@fi': 'kärnkraftverk@sv',
    'utsläpp@fi': 'utsläpp@fi',
    'fjärrvärme@fi': 'fjärrvärme@sv',
    'båtar@fi': 'båtar@sv',
    'kvalitattiiviset menetelmät@fi': 'kvalitatiivinen tutkimus@fi',
    'competition (activity)@en': 'competition (economy)@en',
    'konkurrens@sv': 'konkurrens (ekonomi)@sv',
    'kilpailu (toiminta)@fi': 'kilpailu (talous)@fi',
    'religiositet@fi': 'religiositet@sv',
    'utsläpp@fi': 'utsläpp@sv',
    'ammatinvalinnaohjaus@fi': 'ammatinvalinnanohjaus@fi',
    'våld i närrelation@sv': 'våld i närrelationen@sv',
    'koulukäynti@fi': 'koulunkäynti@fi',
    'narratiivinen tuktimus@fi': 'narratiivinen tutkimus@fi',
    'diskurssinanalyysi@fi': 'diskurssianalyysi@fi',
    'avlöning@sv': 'lönenivå@sv',
    'nanomaterials@sv': 'nanomaterial@sv',
    'international straffrätt@sv': 'internationell straffrätt@sv',
    'biomaterialer@sv': 'biomaterial@sv',
    'ammattilitot@fi': 'ammattiliitot@fi',
    'eritystaide@fi': 'esitystaide@fi',
    'språkinläring@sv': 'språkinlärning@sv',
    'medicine@en': 'medicines@en',
    'maalit@fi': 'maalit (maalaus)@fi',
    'values (cultural objects)@en': 'arvot (käsitykset)@fi',
    'plastic@sv': 'plastic@en',
    'chefer@None': 'förmän@sv',
    'organisaatiorakennefi@fi': 'organisaatiorakenne@fi',
#    'becoming released@en': '@en',
#    'frigörelse@sv': '@sv',
#    'vapautuminen@fi': '@fi',
    'berättelser@fi': 'berättelser@sv',
    'stressi@sv': 'stressi@fi',
    'marknadsplanering@fi': 'marknadsplanering@sv',
    'morfologia@sv': 'morfologia@fi',
    'temperature@sv': 'temperatur@sv',
    'administration (action)@sv': 'administration (action)@en',
}

def get_subject(subj):
    label = subj.text
    language = subj.get('language')
    qualifier = subj.get('qualifier')
    if not qualifier:
        return None
    if 'yso' not in qualifier:
        return None
    lang = infer_language(language, qualifier)
    
    key = f"{label}@{lang}"
    if key in lookup_fixes:
        label, lang = lookup_fixes[key].split('@')
    
    uri = lookup_yso(label, lang)
    #if uri is None:
    #    print(f"    '{label}@{lang}': '@{lang}',")
    return uri

In [13]:
import os, os.path
import shutil

LANGMAP = {'sv': 'swe', 'en': 'eng'}
MIN_SUBJECTS = 3

for langdir in LANGMAP.values():
    if os.path.exists(langdir):
        shutil.rmtree(langdir)
    os.mkdir(langdir)
    os.mkdir(os.path.join(langdir, 'train'))
    os.mkdir(os.path.join(langdir, 'test'))

ndocs = 0
    
files = sorted(glob.glob('abo-theses-page*.xml'))
for fn in files:
    tree = etree.parse(fn)
    items = tree.findall('item')
    for item in items:
        try:
            uri = item.xpath("metadata[@element='identifier'][@qualifier='uri']")[0].text
            did = uri.split('/')[-1]
        except IndexError:
            did = None
        try:
            lang = item.xpath("metadata[@element='language']")[0].text
        except IndexError:
            lang = None
        lang = LANGMAP.get(lang, lang)
        if lang not in LANGMAP.values():
            continue
        files = item.xpath("file")
        if not files:
            continue
        url = files[0].text
        date = item.xpath("metadata[@element='date'][@qualifier='accessioned']")[0].text[:10]
        raw_subjs = item.xpath("metadata[@element='subject']")
        subjects = set(filter(None, [get_subject(subj) for subj in raw_subjs]))
        if len(subjects) >= MIN_SUBJECTS:
            print("\t".join([date, str(did), str(lang), str(url), str(subjects)]))
            ndocs += 1
        if date <= "2021-08-31":
            subset = 'train'
        else:
            subset = 'test'
        urlfn = os.path.join(lang, subset, f"{did}.url")
        with open(urlfn, "w") as urlf:
            print(url, file=urlf)
        tsvfn = os.path.join(lang, subset, f"{did}.tsv")
        with open(tsvfn, "w") as tsvf:
            for uri in subjects:
                print(f"<{uri}>", file=tsvf)



2020-12-14	180129	swe	https://www.doria.fi/bitstream/handle/10024/180129/ekholm_suzanne.pdf	{'http://www.yso.fi/onto/yso/p6333', 'http://www.yso.fi/onto/yso/p1650', 'http://www.yso.fi/onto/yso/p20743', 'http://www.yso.fi/onto/yso/p20240'}
2020-12-16	180160	swe	https://www.doria.fi/bitstream/handle/10024/180160/heinonen_annastina.pdf	{'http://www.yso.fi/onto/yso/p556', 'http://www.yso.fi/onto/yso/p17111', 'http://www.yso.fi/onto/yso/p1650', 'http://www.yso.fi/onto/yso/p10631'}
2020-12-21	180190	swe	https://www.doria.fi/bitstream/handle/10024/180190/hoglund_nico.pdf	{'http://www.yso.fi/onto/yso/p25177', 'http://www.yso.fi/onto/yso/p13643', 'http://www.yso.fi/onto/yso/p9503'}
2021-01-11	180225	swe	https://www.doria.fi/bitstream/handle/10024/180225/smeds_annica.pdf	{'http://www.yso.fi/onto/yso/p2488', 'http://www.yso.fi/onto/yso/p4391', 'http://www.yso.fi/onto/yso/p8543'}
2021-01-11	180223	swe	https://www.doria.fi/bitstream/handle/10024/180223/ostman_karolina.pdf	{'http://www.yso.fi/onto/y

2021-11-29	182698	swe	https://www.doria.fi/bitstream/handle/10024/182698/norrvik_terese.pdf	{'http://www.yso.fi/onto/yso/p19564', 'http://www.yso.fi/onto/yso/p3830', 'http://www.yso.fi/onto/yso/p14387', 'http://www.yso.fi/onto/yso/p9983', 'http://www.yso.fi/onto/yso/p3913'}
2021-11-29	182703	swe	https://www.doria.fi/bitstream/handle/10024/182703/kraufvelin_my.pdf	{'http://www.yso.fi/onto/yso/p6720', 'http://www.yso.fi/onto/yso/p556', 'http://www.yso.fi/onto/yso/p12864', 'http://www.yso.fi/onto/yso/p17536', 'http://www.yso.fi/onto/yso/p1896', 'http://www.yso.fi/onto/yso/p8113', 'http://www.yso.fi/onto/yso/p1066', 'http://www.yso.fi/onto/yso/p8112', 'http://www.yso.fi/onto/yso/p1631'}
2021-11-29	182709	eng	https://www.doria.fi/bitstream/handle/10024/182709/slimani_jasmin.pdf	{'http://www.yso.fi/onto/yso/p1068', 'http://www.yso.fi/onto/yso/p18118', 'http://www.yso.fi/onto/yso/p17111', 'http://www.yso.fi/onto/yso/p29580', 'http://www.yso.fi/onto/yso/p25531', 'http://www.yso.fi/onto/yso/p71

In [12]:
! for set in {swe,eng}/{train,test}; do echo -n "$set "; ls $set/*.tsv|wc -l; done

swe/train 539
swe/test 115
eng/train 346
eng/test 79
